In [7]:
import pandas as pd
import requests
import io

url = 'http://116.85.47.250:9898/dab287fffe388169dffc135f95aa851b/output/result.csv'
csv_data = requests.get(url).content

In [8]:
cvs = pd.read_csv(io.StringIO(csv_data.decode('utf-8')))

In [11]:
cvs.head(4)

,id,label
0,0,3
1,1,0
2,2,2
3,3,6
